# Transforming Predictions into Intensities 


This notebook will create a new categorical variable which converts the actuals and predictions into the following groups: (1) Peaceful; (2) Low-intensity; (3) High-intensity. 

**Peaceful** refers to zero conflict-related deaths in a given country for each month over the past 6-months.

**Low-intensity** refers to 1-24 conflict-related deaths in a given country for each month over the past 6-months.

**High-intensity** refers to 100 or more conflict-related deaths in a given country for each month over the past 6-months.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
import views_mapper2
from views_mapper2.mapper2 import Mapper2
from views_mapper2 import color
from views_mapper2.label_writer import vid2date
from views_mapper2.dictionary_writer import standard_scale

# Packages from Mapper2
from ingester3.ViewsMonth import ViewsMonth
from ingester3.Country import Country
import pycountry
from datetime import date
import os
import numpy as np

# Mapper
import geopandas as gpd

import sqlalchemy as sa
#from ingester3.config import source_db_path

# Other packages
import pickle as pkl

#Parallelization
from joblib import Parallel, delayed, cpu_count
from functools import partial

from pathlib import Path

# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')
import os
from pathlib import Path

from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

from FetchData import FetchData, RetrieveFromList, ReturnQsList, get_df_from_datasets_by_name
from ViewsEstimators import *

In [ ]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
prod_id = '2022_04_t01'
level = 'cm'
months_prior = 3

#note the home path has been automated with the os.path.expanduser function
#no need to adjust the folder as long as ViEWS on the dropbox is the destination

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS_new/'
Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/'
localgitpath = home + '/Desktop/VIEWS_new/'

# Some output that 
if os.getlogin() == 'havardhegre1':
    WriteToOverleaf = True
    overleafpath = f'/Users/havardhegre1/Dropbox (ViEWS)/Apps/Overleaf/VIEWS documentation Fatalities002/Figures/'

##################
print('OS Login is:', os.getlogin())
print('OS path is set to:', home)
print('Local Git path is set to:', localgitpath)
print('Dropbox path is set to:', Mydropbox)

# Importing in Data


    1: Pull in the actuals, restrict the data to EndOfHistory minus 12 mmonth to EndOfHistory

    2: Pull in the future predictions, 1 to 36 months into the future

### Importing the Acutals

In [ ]:
#Fetch the actuals for the End of History form VIEWSER

from viewser import Queryset, Column

# Conflict types are **NOT** logged and **NOT** time lagged (as visualized in the definition below)

qs1 = (Queryset("actuals_monthly", "country_month")
            .with_column(Column("ged_best_sb", from_table= "ged2_cm", from_column="ged_sb_best_sum_nokgi")
                        .transform.missing.fill()
                        )
              
            .with_theme("fatalities")
      
            .describe(""" Fatalities conflict history, cm level
            
            Monthly Update
            
            """)
      
            )

actuals = qs1.publish().fetch()

In [ ]:
# This command resets the DF to include all country-months 18 months prior to EOH
actuals = actuals.loc[EndOfHistory-40:EndOfHistory]
actuals = actuals.reset_index()
actuals['step'] = actuals['month_id'] - EndOfHistory
actuals = actuals.set_index(['step', 'country_id'], drop = True)

# Creates a new column in df with the categorical forms of 'ln_ged_sb_dp'
#THIS WILL LOG IT: actuals['ged_best_sb'] = np.log1p(actuals['ged_best_sb'])
actuals.rename(columns={'ged_best_sb':'fatalities'}, inplace=True)

### Importing the Future Predictions

In [ ]:
# Read in the predictions from the Common Parameters defined above
# Redefine month_id into steps as a index, change the name of some variables for consistency/workability with code

predstore_future = level +  '_' + 'genetic_ensemble' + '_f' + str(EndOfHistory)
predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)

predictions_df = predictions_df.reset_index()
predictions_df['step'] = predictions_df['month_id'] - EndOfHistory
predictions_df = predictions_df.set_index(['step', 'country_id'], drop = True)
predictions_df['step_combined'] = np.expm1(predictions_df['step_combined'])
predictions_df.rename(columns={'step_combined':'fatalities'}, inplace=True)

## Merge the DFs

In [ ]:
merged_df = pd.concat([actuals, predictions_df], axis=0)
display(merged_df)

## Data Manipulation

In this section of code, I will create:

    1. Violence Variable (categorical): 0=Peace; 1= Low violence (x < 100); 2= High violence (x > 100)

    2) Change in Violence Level (integer): The difference in violence level from t-1 to t

    3) Monthly Change in Violence Level (Character): Rapid De-escalation, De-escalation, Status Quo, Escalation, 
       Rapid Escalation

    4) Rolling Mean of Fatalities (integer): The mean of fatalities in the 4-months prior to month t.
    
    5) Intensity Level (categorical): log1p(x) < 0.5 is Peaceful, log1p(x) < 100 is Low Intensity, 
       log1p(x) > 100 is High Intensity 


In [ ]:
# This cell defines a function that creates a multiclass categorical variable grouping fatalities 
# 0 = Peaceful = a country-month without any fatalities
# 1 = low-violence = a country-month with 1-24 fatalities
# 2 = high-violence = a country-month with more than 24 fatalities
    
def ged_fatalities(x):
    if x < 0.5:
        return 0
    elif x < 100:
        return 1
    else:
        return 2
       
def ged_intensity_lvl(x):
    if x < 0.5:
        return 0
    elif x < 100:
        return 1
    else:
        return 2  

In [ ]:
merged_df['fatcat'] = merged_df['fatalities'].apply(ged_fatalities)

merged_df = merged_df.reset_index()

# Create a variable that subtracts the observation in the row above by the row below and prints the output
merged_df['fatcat_change'] = merged_df.groupby('country_id')['fatcat'].diff()

# Fill the top values of ['status'] with zeros
merged_df['fatcat_change'] = merged_df['fatcat_change'].fillna(0)

#actuals = actuals.set_index(['month_id', 'country_id'], drop = True)
merged_df = merged_df.set_index(['step', 'country_id'], drop = True)

#actuals['monthly_change'] = actuals['violence_lvl_change']
merged_df['monthly_change'] = merged_df['fatcat_change']

merged_df = merged_df.reset_index()

In [ ]:
violence_conditions = [
    (merged_df['monthly_change'] == -2), 
    (merged_df['monthly_change'] == -1),
    (merged_df['monthly_change'] == 0),
    (merged_df['monthly_change'] == 1),
    (merged_df['monthly_change'] == 2)
]

violence_categories = ['Rapid Descalation', 'De-escalation', 'Status Quo', 'Escalation', 'Rapid Escalation']

In [ ]:
merged_df['monthly_change'] = np.select(violence_conditions, violence_categories)

merged_df['rolling_mean_violence_4'] = merged_df.groupby('country_id')['fatalities'].transform(lambda x: x.shift(1).rolling(months_prior, 1).mean())

merged_df['intensity_lvl_4'] = merged_df['rolling_mean_violence_4'].apply(ged_intensity_lvl)

#Reset the df to only include 12 months prior
merged_df = merged_df.loc[merged_df['step'] >= -12]

#Reset the index
merged_df = merged_df.set_index(['step', 'country_id'], drop = True)


In [ ]:
display(merged_df)

In [ ]:
merged_df.to_csv('/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/fatcat.csv', sep=';')

# Prediction Uncertainties 

This section will merge in the prediction uncertainties to 'merged_df'

In [ ]:
# read in the future predictions with uncertainty
future = pd.read_csv('/Users/chandlerwilliams/Desktop/VIEWS_new/viewsforecasting/MonthlyUpdates/Categorical_probabilities.csv')
fatcat = pd.read_csv('/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/fatcat.csv', sep = ';')

# set the indexes for merging
fatcat = fatcat.set_index(['month_id', 'country_id'], drop = True)
future = future.set_index(['month_id', 'country_id'], drop = True)


#join the dfs
fatcat_final = fatcat.join(future)

# reset the index to country-step
fatcat_final = fatcat_final.reset_index()
fatcat_final = fatcat_final.set_index(['step', 'country_id'], drop = True)

display(fatcat_final)

In [ ]:
fatcat1 = fatcat_final.copy()

In [ ]:
fatcat1['prob_fatcat_0'] = fatcat1['multi_0_logit']
fatcat1['prob_fatcat_1'] = fatcat1['multi_1_logit'] + fatcat1['multi_2_logit']
fatcat1['prob_fatcat_2'] = fatcat1['multi_3_logit'] + fatcat1['multi_4_logit']

In [ ]:
fatcat1 = fatcat1.reset_index()
fatcat1 = fatcat1.drop(['step_combined', 'multi_0_logit', 'multi_1_logit', 'multi_2_logit', 'multi_3_logit', 'multi_4_logit' ], axis=1)
#fatcat1 = fatcat1.set_index(['step', 'country_id'], drop = True)

In [ ]:
display(fatcat1)

# Creating a list from Step0 and StepX 

In [ ]:
#This gets fatcat_0 set-up

fatcat_0 = fatcat1[fatcat1['step'] == 0] 
fatcat_0 = fatcat_0.sort_values(by=['country_id'], ascending=True)
fatcat_0_pivot = pd.pivot_table(fatcat_0, values=['fatalities', 'fatcat', 'prob_fatcat_0', 'prob_fatcat_1', 'prob_fatcat_2'], 
                                index=['country_id'])

fatcat_0_pivot.rename(columns={'fatalities': 'fatalities_step_0'}, inplace=True)
fatcat_0_pivot.rename(columns={'fatcat': 'fatcat_step_0'}, inplace=True)

display(fatcat_0_pivot)

# This Cell changes the Number of Months ahead

In [ ]:
#This gets fatcat_X set-up
#Get this to be a parameter in the future

fatcat_12 = fatcat1[fatcat1['step'] == 12] 
fatcat_12 = fatcat_12.sort_values(by=['country_id'], ascending=True)
fatcat_12_pivot = pd.pivot_table(fatcat_12, values=['fatalities', 'fatcat', 'prob_fatcat_0', 'prob_fatcat_1', 'prob_fatcat_2'], 
                                index=['country_id'])

fatcat_12_pivot.rename(columns={'fatalities': 'fatalities_step_12'}, inplace=True)
fatcat_12_pivot.rename(columns={'fatcat': 'fatcat_step_12'}, inplace=True)

display(fatcat_12_pivot)

#fatcat_3 = fatcat_3.sort_values(by=['fatalities'], ascending=False)

In [ ]:
fatcat_lists = fatcat_0_pivot.join(fatcat_12_pivot)
fatcat_lists = fatcat_lists.sort_values('fatalities_step_0', ascending=False)
display(fatcat_lists)

In [ ]:
fatcat_lists['fatcat_change'] = fatcat_lists['fatcat_step_12'] - fatcat_lists['fatcat_step_0']
display(fatcat_lists)

In [ ]:
fatcat_lists = fatcat_lists[['fatalities_step_0', 'fatalities_step_12', 
                               'fatcat_step_0', 'fatcat_step_12', 'fatcat_change','prob_fatcat_0', 
                                'prob_fatcat_1', 'prob_fatcat_2']]
fatcat_lists.head(20)

# Joining Country Names to C_id

In [ ]:
def cid2name(country_id):
    """
    Function useful for labeling countries if needed by their name
    Create a string output name by country id
    Uses ingester3 Country as basis
    :param country_id: Views Country ID
    :return: name
    """
    name = Country(country_id).name
    return name

In [ ]:
fatcat_lists = fatcat_lists.reset_index()

c_id = [int(x) for x in fatcat_lists['country_id']] 
country_names = []
for x in (c_id):
        country_names.append(cid2name(x)) 

In [ ]:
fatcat_lists['country'] = country_names
fatcat_lists = fatcat_lists.set_index('country_id', drop = True)

fatcat_lists = fatcat_lists[['country','fatalities_step_0', 'fatalities_step_12', 
                               'fatcat_step_0', 'fatcat_step_12', 'fatcat_change','prob_fatcat_0', 
                                'prob_fatcat_1', 'prob_fatcat_2']]
fatcat_lists.head(10)

In [ ]:
fatcat_lists.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS/test_folder/fatcat_list.md')
print(fatcat_lists.to_markdown())

|   country_id | country                        |   fatalitiles_step_0 |   fatalitiles_step_12 |   fatcat_step_0 |   fatcat_step_12 |   fatcat_change |   prob_fatcat_0 |   prob_fatcat_1 |   prob_fatcat_2 |
|-------------:|:-------------------------------|---------------------:|----------------------:|----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|
|          117 | Ukraine                        |                 4405 |            77.3859    |               2 |                1 |              -1 |     0.00883439  |       0.870301  |     0.120865    |
|           47 | Burkina Faso                   |                  283 |           184.502     |               2 |                2 |               0 |     0.000667617 |       0.746874  |     0.252459    |
|           60 | Iraq                           |                  215 |           248.903     |               2 |                2 |               0 |     0.000355245 |       0.706862  |     0.292782    |
|           79 | Nigeria                        |                  178 |           430.358     |               2 |                2 |               0 |     9.46496e-05 |       0.614568  |     0.385337    |
|          167 | Congo, DRC                     |                  130 |           149.102     |               2 |                2 |               0 |     0.00101729  |       0.771771  |     0.227211    |
|          133 | Afghanistan                    |                  128 |           611.707     |               2 |                2 |               0 |     2.59833e-05 |       0.517922  |     0.482052    |
|           78 | Niger                          |                  117 |            60.5142    |               2 |                1 |              -1 |     0.0085304   |       0.869193  |     0.122276    |
|           50 | Mali                           |                  112 |           115.149     |               2 |                2 |               0 |     0.0019092   |       0.805926  |     0.192165    |
|          120 | Somalia                        |                  110 |           216.959     |               2 |                2 |               0 |     0.000356757 |       0.707142  |     0.292501    |
|          149 | Myanmar                        |                  102 |           224.745     |               2 |                2 |               0 |     0.000343391 |       0.704626  |     0.295031    |
|          220 | Syria                          |                   85 |           226.976     |               1 |                2 |               1 |     0.000458986 |       0.723479  |     0.276062    |
|           69 | Cameroon                       |                   84 |            67.5486    |               1 |                1 |               0 |     0.00593173  |       0.856558  |     0.13751     |
|          222 | Egypt                          |                   58 |            39.3576    |               1 |                1 |               0 |     0.0250889   |       0.891817  |     0.0830943   |
|           70 | Central African Republic       |                   57 |            27.8548    |               1 |                1 |               0 |     0.035868    |       0.89221   |     0.0719217   |
|          137 | Tajikistan                     |                   46 |             1.3901    |               1 |                1 |               0 |     0.89553     |       0.10387   |     0.000600016 |
|          124 | Yemen                          |                   41 |           564.794     |               1 |                2 |               1 |     2.33412e-05 |       0.509811  |     0.490166    |
|          223 | India                          |                   38 |           114.622     |               1 |                2 |               1 |     0.00223109  |       0.813764  |     0.184005    |
|           57 | Ethiopia                       |                   24 |            48.9255    |               1 |                1 |               0 |     0.0109281   |       0.876571  |     0.112501    |
|          136 | Pakistan                       |                   24 |            98.1837    |               1 |                1 |               0 |     0.00279308  |       0.824604  |     0.172603    |
|          162 | Mozambique                     |                   16 |            25.6152    |               1 |                1 |               0 |     0.0510877   |       0.887275  |     0.0616375   |
|          145 | Philippines                    |                   12 |            29.2376    |               1 |                1 |               0 |     0.0401845   |       0.891278  |     0.0685376   |
|           81 | Togo                           |                    8 |             1.07273   |               1 |                1 |               0 |     0.918421    |       0.0811789 |     0.000399641 |
|           28 | Colombia                       |                    6 |             7.26256   |               1 |                1 |               0 |     0.423901    |       0.563938  |     0.0121617   |
|          218 | Israel                         |                    5 |             9.43122   |               1 |                1 |               0 |     0.290401    |       0.689874  |     0.0197251   |
|          155 | Burundi                        |                    4 |             1.81535   |               1 |                1 |               0 |     0.864146    |       0.134929  |     0.000925683 |
|          166 | Congo                          |                    3 |             1.23762   |               1 |                1 |               0 |     0.905233    |       0.0942555 |     0.000511133 |
|           65 | Russia                         |                    3 |             2.53062   |               1 |                1 |               0 |     0.810111    |       0.188271  |     0.00161776  |
|          125 | Armenia                        |                    1 |             0.639931  |               1 |                1 |               0 |     0.947664    |       0.0521428 |     0.000193038 |
|            9 | Brazil                         |                    1 |             4.08387   |               1 |                1 |               0 |     0.639508    |       0.355585  |     0.0049072   |
|          209 | Indonesia                      |                    1 |             2.8553    |               1 |                1 |               0 |     0.776338    |       0.221529  |     0.0021333   |
|          165 | Angola                         |                    1 |             0.730668  |               1 |                1 |               0 |     0.942755    |       0.0570216 |     0.000223595 |
|          150 | Thailand                       |                    0 |             1.67938   |               0 |                1 |               1 |     0.877575    |       0.121646  |     0.000779309 |
|          126 | Azerbaijan                     |                    0 |             1.07526   |               0 |                1 |               1 |     0.922693    |       0.076941  |     0.00036589  |
|          148 | Laos                           |                    0 |             0.0802679 |               0 |                0 |               0 |     0.975861    |       0.0240853 |     5.41491e-05 |
|          123 | Uzbekistan                     |                    0 |             0.0718579 |               0 |                0 |               0 |     0.976051    |       0.0238951 |     5.34452e-05 |
|          135 | Nepal                          |                    0 |             0.063039  |               0 |                0 |               0 |     0.976344    |       0.0236036 |     5.23735e-05 |
|          122 | Turkmenistan                   |                    0 |             0.0836945 |               0 |                0 |               0 |     0.975544    |       0.0244005 |     5.53238e-05 |
|          121 | Sri Lanka                      |                    0 |             0.0787656 |               0 |                0 |               0 |     0.975914    |       0.0240321 |     5.39521e-05 |
|          119 | Oman                           |                    0 |             0.148478  |               0 |                0 |               0 |     0.97258     |       0.0273528 |     6.67914e-05 |
|          152 | Marshall Is.                   |                    0 |             0.0549769 |               0 |                0 |               0 |     0.976877    |       0.0230725 |     5.04423e-05 |
|          118 | Maldives                       |                    0 |             0.121967  |               0 |                0 |               0 |     0.974777    |       0.0251643 |     5.82101e-05 |
|          153 | Micronesia                     |                    0 |             0.0580086 |               0 |                0 |               0 |     0.976797    |       0.0231518 |     5.0729e-05  |
|          151 | Vietnam                        |                    0 |             0.0847558 |               0 |                0 |               0 |     0.975798    |       0.0241479 |     5.43819e-05 |
|          127 | Bahrain                        |                    0 |             0.131199  |               0 |                0 |               0 |     0.974389    |       0.0255511 |     5.96933e-05 |
|          134 | Kyrgyzstan                     |                    0 |             0.685644  |               0 |                1 |               1 |     0.944559    |       0.0552289 |     0.000212163 |
|          128 | Iran                           |                    0 |             2.23198   |               0 |                1 |               1 |     0.824541    |       0.174042  |     0.00141695  |
|          129 | Kuwait                         |                    0 |             0.109144  |               0 |                0 |               0 |     0.975065    |       0.0248779 |     5.7121e-05  |
|          147 | Cambodia                       |                    0 |             0.0545451 |               0 |                0 |               0 |     0.976154    |       0.0237927 |     5.30681e-05 |
|          146 | South Korea                    |                    0 |             0.0784993 |               0 |                0 |               0 |     0.975925    |       0.0240214 |     5.39126e-05 |
|          130 | Qatar                          |                    0 |             0.13714   |               0 |                0 |               0 |     0.974239    |       0.0257006 |     6.02703e-05 |
|          144 | Palau                          |                    0 |             0.0517527 |               0 |                0 |               0 |     0.976974    |       0.0229758 |     5.00937e-05 |
|          143 | North Korea                    |                    0 |             0.277585  |               0 |                0 |               0 |     0.967564    |       0.0323482 |     8.80494e-05 |
|          142 | Japan                          |                    0 |             0.137164  |               0 |                0 |               0 |     0.9743      |       0.0256404 |     6.00377e-05 |
|          140 | Brunei                         |                    0 |             0.0256329 |               0 |                0 |               0 |     0.977655    |       0.0222971 |     4.76739e-05 |
|          139 | Bhutan                         |                    0 |             0.121233  |               0 |                0 |               0 |     0.973783    |       0.0261547 |     6.20366e-05 |
|          131 | Saudi Arabia                   |                    0 |             0.327209  |               0 |                0 |               0 |     0.96419     |       0.0357061 |     0.00010359  |
|          132 | United Arab Emirates           |                    0 |             0.246972  |               0 |                0 |               0 |     0.967978    |       0.0319361 |     8.62099e-05 |
|          138 | Bangladesh                     |                    0 |             0.474989  |               0 |                0 |               0 |     0.955909    |       0.0439448 |     0.000145739 |
|            1 | Guyana                         |                    0 |             0.082997  |               0 |                0 |               0 |     0.974954    |       0.0249884 |     5.75402e-05 |
|          154 | Botswana                       |                    0 |             0.157648  |               0 |                0 |               0 |     0.970716    |       0.0292093 |     7.44253e-05 |
|          231 | Montenegro                     |                    0 |             0.0208817 |               0 |                0 |               0 |     0.977753    |       0.0221993 |     4.73292e-05 |
|          183 | Canada                         |                    0 |             0.0446476 |               0 |                0 |               0 |     0.976991    |       0.0229591 |     5.00338e-05 |
|          184 | Germany                        |                    0 |             0.101797  |               0 |                0 |               0 |     0.973737    |       0.0262012 |     6.22188e-05 |
|          198 | Taiwan                         |                    0 |             0.0804306 |               0 |                0 |               0 |     0.975615    |       0.0243294 |     5.50581e-05 |
|          199 | China                          |                    0 |             0.336448  |               0 |                0 |               0 |     0.962833    |       0.0370565 |     0.000110112 |
|          205 | Malaysia                       |                    0 |             0.215285  |               0 |                0 |               0 |     0.969947    |       0.0299754 |     7.76679e-05 |
|          206 | Singapore                      |                    0 |             0.0524929 |               0 |                0 |               0 |     0.976784    |       0.0231656 |     5.07789e-05 |
|          213 | Libya                          |                    0 |             0.958754  |               0 |                1 |               1 |     0.925025    |       0.0746271 |     0.000347965 |
|          214 | Chad                           |                    0 |             3.62101   |               0 |                1 |               1 |     0.692276    |       0.304023  |     0.00370107  |
|          232 | Kosovo                         |                    0 |             0.159556  |               0 |                0 |               0 |     0.971592    |       0.0283367 |     7.07972e-05 |
|          156 | Rwanda                         |                    0 |             0.28805   |               0 |                0 |               0 |     0.964848    |       0.0350515 |     0.000100483 |
|          233 | Serbia                         |                    0 |             0.027636  |               0 |                0 |               0 |     0.977504    |       0.0224481 |     4.82084e-05 |
|          234 | United States                  |                    0 |             0.234422  |               0 |                0 |               0 |     0.967525    |       0.0323865 |     8.82207e-05 |
|          235 | Uganda                         |                    0 |             1.42339   |               0 |                1 |               1 |     0.894767    |       0.104626  |     0.000607249 |
|          237 | Kenya                          |                    0 |             4.33044   |               0 |                1 |               1 |     0.644919    |       0.350306  |     0.00477505  |
|          242 | Tanzania                       |                    0 |             0.70876   |               0 |                1 |               1 |     0.943942    |       0.0558418 |     0.000216045 |
|          243 | Morocco                        |                    0 |             0.704375  |               0 |                1 |               1 |     0.943135    |       0.0566441 |     0.000221168 |
|          244 | Mauritania                     |                    0 |             0.0651448 |               0 |                0 |               0 |     0.975815    |       0.0241302 |     5.43162e-05 |
|          245 | Sudan                          |                    0 |             2.63372   |               0 |                1 |               1 |     0.798101    |       0.200105  |     0.00179377  |
|          182 | Vanuatu                        |                    0 |             0.0621531 |               0 |                0 |               0 |     0.976554    |       0.0233946 |     5.16103e-05 |
|          181 | Tuvalu                         |                    0 |             0.0552782 |               0 |                0 |               0 |     0.976803    |       0.0231467 |     5.07107e-05 |
|          180 | Solomon Is.                    |                    0 |             0.0662587 |               0 |                0 |               0 |     0.976382    |       0.023566  |     5.22358e-05 |
|          179 | Papua New Guinea               |                    0 |             0.140628  |               0 |                0 |               0 |     0.972402    |       0.0275305 |     6.7508e-05  |
|          157 | Zambia                         |                    0 |             0.250008  |               0 |                0 |               0 |     0.967896    |       0.0320172 |     8.65708e-05 |
|          158 | Zimbabwe                       |                    0 |             0.116806  |               0 |                0 |               0 |     0.973992    |       0.0259467 |     6.12253e-05 |
|          159 | Comoros                        |                    0 |             0.0554144 |               0 |                0 |               0 |     0.976715    |       0.0232343 |     5.10277e-05 |
|          116 | Romania                        |                    0 |             0.0522687 |               0 |                0 |               0 |     0.976399    |       0.0235485 |     5.21719e-05 |
|          161 | Malawi                         |                    0 |             0.101786  |               0 |                0 |               0 |     0.974147    |       0.0257927 |     6.06271e-05 |
|          163 | South Africa                   |                    0 |             0.394185  |               0 |                0 |               0 |     0.958988    |       0.0408824 |     0.000129422 |
|          164 | Swaziland                      |                    0 |             0.141566  |               0 |                0 |               0 |     0.972876    |       0.0270585 |     6.56106e-05 |
|          168 | Fiji                           |                    0 |             0.0713781 |               0 |                0 |               0 |     0.976248    |       0.023699  |     5.27235e-05 |
|          169 | Gabon                          |                    0 |             0.116735  |               0 |                0 |               0 |     0.974085    |       0.0258537 |     6.08635e-05 |
|          170 | Namibia                        |                    0 |             0.0865763 |               0 |                0 |               0 |     0.97463     |       0.0253117 |     5.87732e-05 |
|          171 | New Zealand                    |                    0 |             0.154933  |               0 |                0 |               0 |     0.971766    |       0.0281639 |     7.0087e-05  |
|          172 | Madagascar                     |                    0 |             0.122414  |               0 |                0 |               0 |     0.973342    |       0.0265942 |     6.37649e-05 |
|          173 | Mauritius                      |                    0 |             0.0317865 |               0 |                0 |               0 |     0.977361    |       0.0225901 |     4.87128e-05 |
|          174 | Seychelles                     |                    0 |             0.0646196 |               0 |                0 |               0 |     0.976511    |       0.0234375 |     5.17667e-05 |
|          176 | Timor Leste                    |                    0 |             0.0705129 |               0 |                0 |               0 |     0.976336    |       0.0236114 |     5.24022e-05 |
|          177 | Australia                      |                    0 |             0.107208  |               0 |                0 |               0 |     0.975179    |       0.024764  |     5.669e-05   |
|          178 | Nauru                          |                    0 |             0.0581361 |               0 |                0 |               0 |     0.976715    |       0.0232344 |     5.10283e-05 |
|          160 | Lesotho                        |                    0 |             0.0287119 |               0 |                0 |               0 |     0.977455    |       0.0224963 |     4.83793e-05 |
|          109 | Switzerland                    |                    0 |             0.128972  |               0 |                0 |               0 |     0.972425    |       0.0275077 |     6.74159e-05 |
|          115 | Moldova                        |                    0 |             0.0236666 |               0 |                0 |               0 |     0.977665    |       0.0222874 |     4.76398e-05 |
|           52 | Senegal                        |                    0 |             0.197265  |               0 |                0 |               0 |     0.969647    |       0.0302739 |     7.89462e-05 |
|           27 | Belize                         |                    0 |             0.0962555 |               0 |                0 |               0 |     0.973974    |       0.0259646 |     6.12948e-05 |
|           29 | Costa Rica                     |                    0 |             0.0640181 |               0 |                0 |               0 |     0.975445    |       0.0244995 |     5.56946e-05 |
|           30 | Cuba                           |                    0 |             0.0339584 |               0 |                0 |               0 |     0.977304    |       0.0226472 |     4.89163e-05 |
|           31 | El Salvador                    |                    0 |             0.0721047 |               0 |                0 |               0 |     0.975782    |       0.0241636 |     5.44402e-05 |
|           32 | Honduras                       |                    0 |             0.175023  |               0 |                0 |               0 |     0.971234    |       0.0286938 |     7.22733e-05 |
|           33 | Nicaragua                      |                    0 |             0.0797335 |               0 |                0 |               0 |     0.975361    |       0.0245829 |     5.60079e-05 |
|           34 | Panama                         |                    0 |             0.0933663 |               0 |                0 |               0 |     0.974859    |       0.0250834 |     5.79017e-05 |
|           35 | Antigua and Barbuda            |                    0 |             0.0380064 |               0 |                0 |               0 |     0.977272    |       0.0226794 |     4.90313e-05 |
|           36 | St. Kitts and Nevis            |                    0 |             0.0532599 |               0 |                0 |               0 |     0.976946    |       0.0230035 |     5.01936e-05 |
|           37 | Iceland                        |                    0 |             0.0273508 |               0 |                0 |               0 |     0.977491    |       0.0224611 |     4.82545e-05 |
|           38 | Ireland                        |                    0 |             0.370563  |               0 |                0 |               0 |     0.966007    |       0.0338978 |     9.50988e-05 |
|           39 | United Kingdom                 |                    0 |             0.0355114 |               0 |                0 |               0 |     0.977284    |       0.0226671 |     4.89875e-05 |
|           40 | Cape Verde                     |                    0 |             0.0765047 |               0 |                0 |               0 |     0.976155    |       0.0237915 |     5.30635e-05 |
|           41 | Cote d'Ivoire                  |                    0 |             0.396334  |               0 |                0 |               0 |     0.960659    |       0.0392202 |     0.000120884 |
|           42 | Ghana                          |                    0 |             0.179056  |               0 |                0 |               0 |     0.969928    |       0.0299944 |     7.77492e-05 |
|           43 | Liberia                        |                    0 |             0.0270044 |               0 |                0 |               0 |     0.977494    |       0.0224575 |     4.82415e-05 |
|           45 | Portugal                       |                    0 |             0.0413575 |               0 |                0 |               0 |     0.977097    |       0.0228535 |     4.96545e-05 |
|           46 | Spain                          |                    0 |             0.154145  |               0 |                0 |               0 |     0.971187    |       0.0287403 |     7.24664e-05 |
|           48 | Guinea                         |                    0 |             0.203135  |               0 |                0 |               0 |     0.970651    |       0.0292742 |     7.46978e-05 |
|           26 | Bahamas                        |                    0 |             0.0375493 |               0 |                0 |               0 |     0.97729     |       0.0226609 |     4.89654e-05 |
|           25 | Jamaica                        |                    0 |             0.0509799 |               0 |                0 |               0 |     0.976659    |       0.0232896 |     5.12283e-05 |
|           24 | Haiti                          |                    0 |             0.163985  |               0 |                0 |               0 |     0.970504    |       0.029421  |     7.53161e-05 |
|           12 | Paraguay                       |                    0 |             0.17502   |               0 |                0 |               0 |     0.970126    |       0.0297968 |     7.69075e-05 |
|            3 | Trinidad and Tobago            |                    0 |             0.0424617 |               0 |                0 |               0 |     0.976928    |       0.0230216 |     5.0259e-05  |
|            4 | Venezuela                      |                    0 |             0.858276  |               0 |                1 |               1 |     0.936113    |       0.0636192 |     0.000267666 |
|            5 | Samoa                          |                    0 |             0.0675283 |               0 |                0 |               0 |     0.976445    |       0.0235031 |     5.20059e-05 |
|            6 | Tonga                          |                    0 |             0.066812  |               0 |                0 |               0 |     0.976483    |       0.0234653 |     5.1868e-05  |
|            7 | Argentina                      |                    0 |             0.115405  |               0 |                0 |               0 |     0.973383    |       0.0265536 |     6.36044e-05 |
|            8 | Bolivia                        |                    0 |             0.0916558 |               0 |                0 |               0 |     0.973721    |       0.0262163 |     6.22777e-05 |
|           10 | Chile                          |                    0 |             0.049287  |               0 |                0 |               0 |     0.976487    |       0.023461  |     5.18523e-05 |
|           11 | Ecuador                        |                    0 |             0.105865  |               0 |                0 |               0 |     0.972884    |       0.02705   |     6.55767e-05 |
|           13 | Peru                           |                    0 |             0.3846    |               0 |                0 |               0 |     0.959144    |       0.0407279 |     0.000128618 |
|           23 | Dominican Republic             |                    0 |             0.0303072 |               0 |                0 |               0 |     0.977396    |       0.0225556 |     4.859e-05   |
|           14 | Uruguay                        |                    0 |             0.0853119 |               0 |                0 |               0 |     0.974718    |       0.025224  |     5.8438e-05  |
|           16 | Guatemala                      |                    0 |             0.100531  |               0 |                0 |               0 |     0.973929    |       0.0260097 |     6.14705e-05 |
|           17 | Mexico                         |                    0 |             0.899633  |               0 |                1 |               1 |     0.931921    |       0.0677823 |     0.000297055 |
|           18 | Barbados                       |                    0 |             0.0271263 |               0 |                0 |               0 |     0.977533    |       0.0224186 |     4.81036e-05 |
|           19 | Dominica                       |                    0 |             0.0681653 |               0 |                0 |               0 |     0.976403    |       0.0235446 |     5.21577e-05 |
|           20 | Grenada                        |                    0 |             0.0362277 |               0 |                0 |               0 |     0.97734     |       0.0226112 |     4.8788e-05  |
|           21 | St. Lucia                      |                    0 |             0.0394415 |               0 |                0 |               0 |     0.977235    |       0.0227155 |     4.91601e-05 |
|           22 | St. Vincent and the Grenadines |                    0 |             0.0813328 |               0 |                0 |               0 |     0.976065    |       0.0238814 |     5.33947e-05 |
|           49 | Guinea-Bissau                  |                    0 |             0.315359  |               0 |                0 |               0 |     0.965386    |       0.0345161 |     9.797e-05   |
|           53 | Sierra Leone                   |                    0 |             0.0228679 |               0 |                0 |               0 |     0.977665    |       0.0222875 |     4.76402e-05 |
|          114 | Lithuania                      |                    0 |             0.0277672 |               0 |                0 |               0 |     0.977554    |       0.0223979 |     4.80305e-05 |
|           54 | The Gambia                     |                    0 |             0.154187  |               0 |                0 |               0 |     0.971152    |       0.0287753 |     7.2612e-05  |
|           93 | Greece                         |                    0 |             0.0490897 |               0 |                0 |               0 |     0.976399    |       0.0235491 |     5.21741e-05 |
|           94 | Lebanon                        |                    0 |             0.517831  |               0 |                1 |               1 |     0.954424    |       0.0454218 |     0.000153876 |
|           96 | Turkey                         |                    0 |             3.22387   |               0 |                1 |               1 |     0.750241    |       0.247183  |     0.0025758   |
|           97 | Austria                        |                    0 |             0.120722  |               0 |                0 |               0 |     0.972721    |       0.0272128 |     6.62286e-05 |
|           98 | Czech Republic                 |                    0 |             0.0175427 |               0 |                0 |               0 |     0.977857    |       0.0220961 |     4.69663e-05 |
|           99 | Denmark                        |                    0 |             0.0705978 |               0 |                0 |               0 |     0.976276    |       0.0236713 |     5.26218e-05 |
|          100 | Hungary                        |                    0 |             0.10238   |               0 |                0 |               0 |     0.973852    |       0.0260861 |     6.17685e-05 |
|          101 | Poland                         |                    0 |             0.0766887 |               0 |                0 |               0 |     0.975241    |       0.0247026 |     5.64586e-05 |
|          102 | Slovakia                       |                    0 |             0.0212262 |               0 |                0 |               0 |     0.977751    |       0.0222013 |     4.73361e-05 |
|          103 | Slovenia                       |                    0 |             0.0169437 |               0 |                0 |               0 |     0.977885    |       0.0220681 |     4.68679e-05 |
|          104 | Belgium                        |                    0 |             0.0401355 |               0 |                0 |               0 |     0.977067    |       0.0228835 |     4.97619e-05 |
|          105 | France                         |                    0 |             0.0564772 |               0 |                0 |               0 |     0.976539    |       0.0234092 |     5.16633e-05 |
|          107 | Luxembourg                     |                    0 |             0.0683268 |               0 |                0 |               0 |     0.9763      |       0.0236477 |     5.25353e-05 |
|          108 | Netherlands                    |                    0 |             0.0440252 |               0 |                0 |               0 |     0.97701     |       0.0229396 |     4.99638e-05 |
|            2 | Suriname                       |                    0 |             0.0817914 |               0 |                0 |               0 |     0.974838    |       0.0251039 |     5.79797e-05 |
|          110 | Belarus                        |                    0 |             0.0393793 |               0 |                0 |               0 |     0.977032    |       0.0229183 |     4.98871e-05 |
|          111 | Estonia                        |                    0 |             0.0265565 |               0 |                0 |               0 |     0.97759     |       0.022362  |     4.79032e-05 |
|          112 | Finland                        |                    0 |             0.130537  |               0 |                0 |               0 |     0.973215    |       0.0267208 |     6.42664e-05 |
|          113 | Latvia                         |                    0 |             0.0278789 |               0 |                0 |               0 |     0.977541    |       0.0224111 |     4.8077e-05  |
|           92 | Georgia                        |                    0 |             0.0874001 |               0 |                0 |               0 |     0.9754      |       0.0245445 |     5.58635e-05 |
|           90 | Cyprus                         |                    0 |             0.037412  |               0 |                0 |               0 |     0.977232    |       0.0227186 |     4.91714e-05 |
|           89 | Bulgaria                       |                    0 |             0.0505735 |               0 |                0 |               0 |     0.976375    |       0.0235731 |     5.2262e-05  |
|           73 | Tunisia                        |                    0 |             0.294954  |               0 |                0 |               0 |     0.964694    |       0.0352043 |     0.000101205 |
|           55 | Djibouti                       |                    0 |             0.252473  |               0 |                0 |               0 |     0.967407    |       0.0325044 |     8.87503e-05 |
|           56 | Eritrea                        |                    0 |             0.128511  |               0 |                0 |               0 |     0.974264    |       0.0256759 |     6.01749e-05 |
|           58 | Mongolia                       |                    0 |             0.174885  |               0 |                0 |               0 |     0.971442    |       0.0284865 |     7.14148e-05 |
|           62 | Jordan                         |                    0 |             0.117367  |               0 |                0 |               0 |     0.974451    |       0.0254896 |     5.94563e-05 |
|           63 | Kazakhstan                     |                    0 |             0.146329  |               0 |                0 |               0 |     0.973782    |       0.0261559 |     6.20415e-05 |
|           64 | Norway                         |                    0 |             0.104905  |               0 |                0 |               0 |     0.973974    |       0.0259647 |     6.12951e-05 |
|           66 | Sweden                         |                    0 |             0.0370577 |               0 |                0 |               0 |     0.977189    |       0.0227617 |     4.93255e-05 |
|           67 | Algeria                        |                    0 |             1.06863   |               0 |                1 |               1 |     0.920745    |       0.0788738 |     0.000381136 |
|           74 | Benin                          |                    0 |             1.37138   |               0 |                1 |               1 |     0.891608    |       0.107754  |     0.00063755  |
|           87 | Malta                          |                    0 |             0.0693044 |               0 |                0 |               0 |     0.976333    |       0.0236145 |     5.24133e-05 |
|           76 | Equatorial Guinea              |                    0 |             0.148593  |               0 |                0 |               0 |     0.97297     |       0.0269648 |     6.52365e-05 |
|           77 | Kiribati                       |                    0 |             0.068424  |               0 |                0 |               0 |     0.976458    |       0.0234905 |     5.19601e-05 |
|           80 | Sao Tome and Principe          |                    0 |             0.0556217 |               0 |                0 |               0 |     0.976667    |       0.0232821 |     5.12011e-05 |
|           82 | Albania                        |                    0 |             0.0270915 |               0 |                0 |               0 |     0.977487    |       0.0224647 |     4.8267e-05  |
|           83 | Bosnia and Herzegovina         |                    0 |             0.0210291 |               0 |                0 |               0 |     0.977744    |       0.0222089 |     4.73629e-05 |
|           84 | Croatia                        |                    0 |             0.0210603 |               0 |                0 |               0 |     0.977765    |       0.0221879 |     4.72891e-05 |
|           85 | Italy                          |                    0 |             0.0334844 |               0 |                0 |               0 |     0.977205    |       0.0227455 |     4.92675e-05 |
|           86 | Macedonia                      |                    0 |             0.0221054 |               0 |                0 |               0 |     0.977717    |       0.0222359 |     4.74582e-05 |
|          246 | South Sudan                    |                    0 |             3.47229   |               0 |                1 |               1 |     0.721515    |       0.275377  |     0.0031084   |

# Visuals

    1. Countries Step0 and Step12 (escalating)
    2. Countries Step0 and step12 (de-escalating)
    3. Countries Step0 and Step6 (escalating) 
    4. Countries Step0 and Step6 (de-escalating)
    
  

In [ ]:
# 1. Countries Step0 and Step12 (escalating)

fatcat_12_e = fatcat_lists.copy()
fatcat_12_e = fatcat_12_e.loc[fatcat_12_e['fatcat_change'] >= 1 ] 
fatcat_12_e = fatcat_12_e.sort_values(by = ['fatalities_step_12'], ascending=[False])
fatcat_12_e.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS/test_folder/fatcat_12_e.md')
print(fatcat_12_e.to_markdown())


|   Country Id | Country     |   Fatalities (step 0) |   Fatalities (step 12) |   Category (step 0) |   Category (step 12) |   Category Change |   Probability (Peace) |   Probability (Low-violent) |    Probability (High-violent) |
|-------------:|:------------|--------------------:|---------------------:|----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|
|          124 | Yemen       |                  41 |           564.794    |               Low-violence |                High-violence |               Escalation |     2.33412e-05 |       0.509811  |     0.490166    |
|          220 | Syria       |                  85 |           226.976    |               Low-violence |                High-violence |               Escalation |     0.000458986 |       0.723479  |     0.276062    |
|          223 | India       |                  38 |           114.622    |               Low-violence |                High-violence|               Escalation |     0.00223109  |       0.813764  |     0.184005    |
|          237 | Kenya       |                   0 |             4.33044  |               Peace |                Low-violence |               Escalation |     0.644919    |       0.350306  |     0.00477505  |
|          214 | Chad        |                   0 |             3.62101  |               Peace |                Low-violence |               Escalation |     0.692276    |       0.304023  |     0.00370107  |
|          246 | South Sudan |                   0 |             3.47229  |               Peace |                Low-violence |               Escalation |     0.721515    |       0.275377  |     0.0031084   |
|           96 | Turkey      |                   0 |             3.22387  |               Peace |                Low-violence |               Escalation |     0.750241    |       0.247183  |     0.0025758   |
|          245 | Sudan       |                   0 |             2.63372  |               Peace |                Low-violence |               Escalation |     0.798101    |       0.200105  |     0.00179377  |
|          128 | Iran        |                   0 |             2.23198  |               Peace |                Low-violence |               Escalation |     0.824541    |       0.174042  |     0.00141695  |
|          150 | Thailand    |                   0 |             1.67938  |               Peace |                Low-violence |               Escalation |     0.877575    |       0.121646  |     0.000779309 |
|          235 | Uganda      |                   0 |             1.42339  |               Peace |                Low-violence |               Escalation |     0.894767    |       0.104626  |     0.000607249 |
|           74 | Benin       |                   0 |             1.37138  |               Peace |                Low-violence |               Escalation |     0.891608    |       0.107754  |     0.00063755  |
|          126 | Azerbaijan  |                   0 |             1.07526  |               Peace |                Low-violence |               Escalation |     0.922693    |       0.076941  |     0.00036589  |
|           67 | Algeria     |                   0 |             1.06863  |               Peace |                Low-violence |               Escalation |     0.920745    |       0.0788738 |     0.000381136 |
|          213 | Libya       |                   0 |             0.958754 |               Peace |                Low-violence |               Escalation |     0.925025    |       0.0746271 |     0.000347965 |
|           17 | Mexico      |                   0 |             0.899633 |               Peace |                Low-violence |               Escalation |     0.931921    |       0.0677823 |     0.000297055 |
|            4 | Venezuela   |                   0 |             0.858276 |               Peace |                Low-violence |               Escalation |     0.936113    |       0.0636192 |     0.000267666 |
|          242 | Tanzania    |                   0 |             0.70876  |               Peace |                Low-violence |               Escalation |     0.943942    |       0.0558418 |     0.000216045 |
|          243 | Morocco     |                   0 |             0.704375 |               Peace |                Low-violence |               Escalation |     0.943135    |       0.0566441 |     0.000221168 |
|          134 | Kyrgyzstan  |                   0 |             0.685644 |               Peace |                Low-violence |               Escalation |     0.944559    |       0.0552289 |     0.000212163 |
|           94 | Lebanon     |                   0 |             0.517831 |               Peace |                Low-violence |               Escalation |     0.954424    |       0.0454218 |     0.000153876 |

In [ ]:
# 2. Countries Step0 and Step12 (de-escalating)

fatcat_12_d = fatcat_lists.copy()
fatcat_12_d = fatcat_12_d.loc[fatcat_12_d['fatcat_change'] <= -1 ] 
fatcat_12_d = fatcat_12_d.sort_values(by = ['fatalities_step_12'], ascending=[False])
fatcat_12_d.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS/test_folder/fatcat_12_d.md')
print(fatcat_12_d.to_markdown())

|   country_id | country   |   fatalities_step_0 |   fatalities_step_12 |   fatcat_step_0 |   fatcat_step_12 |   fatcat_change |   prob_fatcat_0 |   prob_fatcat_1 |   prob_fatcat_2 |
|-------------:|:----------|--------------------:|---------------------:|----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|
|          117 | Ukraine   |                4405 |              77.3859 |               2 |                1 |              -1 |      0.00883439 |        0.870301 |        0.120865 |
|           78 | Niger     |                 117 |              60.5142 |               2 |                1 |              -1 |      0.0085304  |        0.869193 |        0.122276 |

In [ ]:
# 3. Countries Step0 and Step13 (status Quo)

fatcat_12_sq = fatcat_lists.copy()
fatcat_12_sq = fatcat_12_sq.loc[fatcat_12_sq['fatcat_change'] == 0 ] 
fatcat_12_sq = fatcat_12_sq.loc[fatcat_12_sq['fatalities_step_12'] >= 0.5 ] 
fatcat_12_sq = fatcat_12_sq.sort_values(by = ['fatalities_step_12'], ascending=[False])
fatcat_12_sq.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS/test_folder/fatcat_12_sq.md')
print(fatcat_12_sq.to_markdown())

|   country_id | country                  |   fatalities_step_0 |   fatalities_step_12 |   fatcat_step_0 |   fatcat_step_12 |   fatcat_change |   prob_fatcat_0 |   prob_fatcat_1 |   prob_fatcat_2 |
|-------------:|:-------------------------|--------------------:|---------------------:|----------------:|-----------------:|----------------:|----------------:|----------------:|----------------:|
|          133 | Afghanistan              |                 128 |           611.707    |               2 |                2 |               0 |     2.59833e-05 |       0.517922  |     0.482052    |
|           79 | Nigeria                  |                 178 |           430.358    |               2 |                2 |               0 |     9.46496e-05 |       0.614568  |     0.385337    |
|           60 | Iraq                     |                 215 |           248.903    |               2 |                2 |               0 |     0.000355245 |       0.706862  |     0.292782    |
|          149 | Myanmar                  |                 102 |           224.745    |               2 |                2 |               0 |     0.000343391 |       0.704626  |     0.295031    |
|          120 | Somalia                  |                 110 |           216.959    |               2 |                2 |               0 |     0.000356757 |       0.707142  |     0.292501    |
|           47 | Burkina Faso             |                 283 |           184.502    |               2 |                2 |               0 |     0.000667617 |       0.746874  |     0.252459    |
|          167 | Congo, DRC               |                 130 |           149.102    |               2 |                2 |               0 |     0.00101729  |       0.771771  |     0.227211    |
|           50 | Mali                     |                 112 |           115.149    |               2 |                2 |               0 |     0.0019092   |       0.805926  |     0.192165    |
|          136 | Pakistan                 |                  24 |            98.1837   |               1 |                1 |               0 |     0.00279308  |       0.824604  |     0.172603    |
|           69 | Cameroon                 |                  84 |            67.5486   |               1 |                1 |               0 |     0.00593173  |       0.856558  |     0.13751     |
|           57 | Ethiopia                 |                  24 |            48.9255   |               1 |                1 |               0 |     0.0109281   |       0.876571  |     0.112501    |
|          222 | Egypt                    |                  58 |            39.3576   |               1 |                1 |               0 |     0.0250889   |       0.891817  |     0.0830943   |
|          145 | Philippines              |                  12 |            29.2376   |               1 |                1 |               0 |     0.0401845   |       0.891278  |     0.0685376   |
|           70 | Central African Republic |                  57 |            27.8548   |               1 |                1 |               0 |     0.035868    |       0.89221   |     0.0719217   |
|          162 | Mozambique               |                  16 |            25.6152   |               1 |                1 |               0 |     0.0510877   |       0.887275  |     0.0616375   |
|          218 | Israel                   |                   5 |             9.43122  |               1 |                1 |               0 |     0.290401    |       0.689874  |     0.0197251   |
|           28 | Colombia                 |                   6 |             7.26256  |               1 |                1 |               0 |     0.423901    |       0.563938  |     0.0121617   |
|            9 | Brazil                   |                   1 |             4.08387  |               1 |                1 |               0 |     0.639508    |       0.355585  |     0.0049072   |
|          209 | Indonesia                |                   1 |             2.8553   |               1 |                1 |               0 |     0.776338    |       0.221529  |     0.0021333   |
|           65 | Russia                   |                   3 |             2.53062  |               1 |                1 |               0 |     0.810111    |       0.188271  |     0.00161776  |
|          155 | Burundi                  |                   4 |             1.81535  |               1 |                1 |               0 |     0.864146    |       0.134929  |     0.000925683 |
|          137 | Tajikistan               |                  46 |             1.3901   |               1 |                1 |               0 |     0.89553     |       0.10387   |     0.000600016 |
|          166 | Congo                    |                   3 |             1.23762  |               1 |                1 |               0 |     0.905233    |       0.0942555 |     0.000511133 |
|           81 | Togo                     |                   8 |             1.07273  |               1 |                1 |               0 |     0.918421    |       0.0811789 |     0.000399641 |
|          165 | Angola                   |                   1 |             0.730668 |               1 |                1 |               0 |     0.942755    |       0.0570216 |     0.000223595 |
|          125 | Armenia                  |                   1 |             0.639931 |               1 |                1 |               0 |     0.947664    |       0.0521428 |     0.000193038 |

In [ ]:
# Playing here down

In [ ]:
fatcat_12_e_plot = fatcat_12_e.copy()
fatcat_12_e_plot = fatcat_12_e.reset_index()

# The horizontal plot is made using the hline function
plt.hlines(y=fatcat_12_e_plot['country_id'], xmin=fatcat_12_e_plot['fatalities_step_0'], xmax=fatcat_12_e_plot['fatalities_step_12'], color='grey', alpha=0.4)
plt.scatter(fatcat_12_e_plot['fatalities_step_0'], fatcat_12_e_plot['country_id'], color='skyblue', alpha=1, label='Step 0')
plt.scatter(fatcat_12_e_plot['fatalities_step_12'], fatcat_12_e_plot['country_id'], color='green', alpha=0.4 , label='Step 12')
plt.legend()
 
# Add title and axis names
#plt.yticks(my_range, ordered_df['group'])
#plt.title("Comparison of the value 1 and the value 2", loc='left')
#plt.xlabel('Value of the variables')
#plt.ylabel('Group')

# Show the graph
plt.show()

In [ ]:
fatcat_12_e_plot = fatcat_12_e.copy()

In [ ]:
#conda install statsmodels

In [ ]:
from sklearn import linear_model

df = fatcat_12_e_plot

## use linear model to determine outliers by residual
X = df["fatalities_step_0"].values.reshape(-1, 1)
y = df["fatalities_step_12"].values

regr = linear_model.LinearRegression()
regr.fit(X, y)

df["predicted_0"] = regr.predict(X)
df["residual"] = df["fatalities_step_12"] - df["predicted_0"]
residual_mean, residual_std = df["residual"].mean(), df["residual"].std()
df["residual_normalized"] = (((df["fatalities_step_12"] - df["predicted_0"]) - residual_mean) / residual_std).abs()

## determine outliers using whatever method you like
outliers = df.loc[df["residual_normalized"] > 3.0, ["fatalities_step_0","fatalities_step_12"]]

fig = px.scatter(df, x="fatalities_step_0", y="fatalities_step_12", text= 'country', trendline="ols", trendline_color_override="red")

## add text to outliers using their (x,y) coordinates:
for x,y in outliers.itertuples(index=False):
    fig.add_annotation(
        x=x, y=y,
        text="outlier",
        showarrow=False,
        yshift=15
    )
fig.show()



In [ ]:
import plotly.express as px
import statsmodels as sm
fig = px.scatter(fatcat_12_e_plot, x="fatalities_step_12", y="fatalities_step_0", trendline="ols", 
                text = 'country')
fig.update_xaxes(title_text='Number of Fatalities in 12 Months')
fig.update_yaxes(title_text='Number of Fatalities at Present')

fig.show()
plt.savefig('/Users/chandlerwilliams/Desktop/VIEWS/test_folder/step_change.png')



In [ ]:
fatcat_12_e_plot = fatcat_lists.reset_index()
df3 = fatcat_12_e_plot.sort_values(by='fatcat_step_0')
my_range=range(1,len(df.index)+1)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))

plt.hlines(y=my_range, xmin=df3['fatcat_step_0'], xmax=df3['fatcat_step_12'], color='grey', alpha=0.1)
plt.scatter(df3['fatcat_step_0'], my_range, color='navy', alpha=1, label='Step 0')
plt.scatter(df3['fatcat_step_12'], my_range, color='gold', alpha=0.8 , label='Step 12')
plt.legend()

# Add title and axis names
plt.yticks(my_range, df3['country_id'])
plt.title("Comparison of Fatalities Category at Step 0 and 12", loc='left')
plt.xlabel('Fatalities Category')
plt.ylabel('Country ID')
